# Analisi delle reti sociali

<img style="float: right;" src="figures/karate_kid.jpg">

*[Zachary's karate club](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) is a well-known social network of a university karate club described in the paper [An Information Flow Model for Conflict and Fission in Small Groups](http://www1.ind.ku.dk/complexLearning/zachary1977.pdf) by Wayne W. Zachary.*

## Indice

1. [Rappresentazione di reti binarie indirette](#reti_binarie_indirette)<br>
2. [Algoritimi di *Force-Direction Placements*](#placements)<br>
3. [Matrice di adiacenza](#adiacenza)<br>
4. [Informazioni di nodo](#informazioni_nodo)<br>
5. [Il cammino più corto (*shortest path*)](#shortest_path)<br>
6. [Utili indici descrittivi](#indici)<br>
    6.1 [Indici descrittivi a livello di nodo](#indici_nodo)<br>
    6.2 [Indici descrittivi a livello di rete](#indici_rete)<br>
7. [Community Detection](#detection)<br>
    7.1 [Metodo di Louvain](#louvain)<br>

# 1. Rappresentazione di reti binarie indirette <a id=reti_binarie_indirette> </a>

In [ ]:
from matplotlib.cbook import mplDeprecation
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=mplDeprecation)

### Grafi

**Grafi**, $\mathcal{G} = (\mathcal{N}, \mathcal{A})$

In [ ]:
G = nx.karate_club_graph()
print(nx.info(G))

* Insieme dei nodi $\mathcal{N} = \{1, \dots, V\}$;

### Nodi

In [ ]:
N = G.nodes()
print("Nodi:", N)

* Un arco è definito come una coppia $\{\{i, j\} : i, j \in \mathcal{N}, i > j\}$;
* Insieme degli archi $\mathcal{A} \subseteq \{\{i, j\} : i, j \in \mathcal{N}, i > j\}$.

### Archi

In [ ]:
A = G.edges()
print("Archi:", A)

# 2. Algoritimi di *Force-Direction Placements* <a id=placements> </a>

Definiscono le posizioni dei nodi utilizzando solo le informazioni sugli archi nella rete con un'interpretazione che deriva dalla **Fisica**. Come?
* **I nodi** sono visti come particelle in un sistema fisco con una certa energia che risulta da due principali forze che agiscono su ogni nodo;
* **Forza repulsiva**: Simile alla forza elettrostatica di **Coulomb**. Agisce su tutti i nodi e genera più energia tanto più i nodi sono vicini;
* **Forza attrattiva**: Simile alla forza della molla di **Hooke**. Agisce solo su nodi connessi e genera più energia tanto più i nodi sono lontani.

Questi algoritmi individuano le posizioni spaziali dei nodi in modo da ottenere la configurazione più stabile nel sistema di particelle (a minor energia).

[spring_layout](https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html): *Position nodes using Fruchterman-Reingold force-directed algorithm.*

In [ ]:
pos = nx.spring_layout(G, iterations=50, seed=3)

In [ ]:
node_color_club = ['lightblue' if (N[nodo]['club'] == "Mr. Hi") else 'orange' for nodo in N]

nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    node_color=node_color_club
)
plt.axis('off')
plt.tight_layout()
plt.show()

# 3. Matrice di adiacenza <a id=adiacenza> </a>

**Matrice di adiacenza**, $Y$
* $Y$ matrice quadrata simmetrica di dimensioni $V \times V$;
* Nodi disposti in riga e colonna;
* $Y_{ij} = Y_{ji} = 1$ se ${i, j} \in \mathcal{A}$ ($i$ e $j$ sono connessi), $0$ altrimenti.

In [ ]:
Y = nx.adjacency_matrix(G)
V, V = Y.shape

In [ ]:
print("Dimensioni: {} X {}".format(V, V))

In [ ]:
print(Y.todense())

### Esempio

Calcolare la matrice di adiacenza $Y$ a partire dal grafo $G$ (vedi definizione sopra).

In [ ]:
from scipy.sparse.csr import csr_matrix

row_ind = []
col_ind = []

for arco in A:
    row_ind.extend(list(arco)) # a, b
    col_ind.extend(list(arco)[::-1]) # b, a
    
Y = csr_matrix((np.ones(len(row_ind)), (np.array(row_ind), np.array(col_ind))), 
               shape=(V, V), dtype=np.int64)

print("Dimensioni: {} X {}\n".format(V, V))
print(Y.todense())

# 4. Informazioni di nodo <a id=informazioni_nodo> </a>

### club

In [ ]:
mr_hi = 0
officer = 33
print("Il nodo {} rappresenta \"Mr. Hi\"".format(mr_hi))
print(N[mr_hi])
print("\nIl nodo {} rappresenta \"Officer\"".format(officer))
print(N[officer])
print("\nI rimanenti nodi appartengono a uno dei due club.")

In [ ]:
node_color_club[mr_hi] = 'blue'
node_color_club[officer] = 'red'

nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    node_color=node_color_club
)
plt.axis('off')
plt.tight_layout()
plt.show()

# 5. Il cammino più corto (*shortest path*) <a id=shortest_path> </a>

* Per ogni coppia di nodi $i$ e $j$ gli *shortest paths* sono i cammini più corti tra nodi interconnessi che uniscono $i$ a $j$;
* Possono essere molteplici;
* Lunghezza dello *shortest path*: numero di archi di cui si compone.

### Esercizio

Utilizzare le funzioni `shortest_path()` e `shortest_path_length()` della libreria NetworkX (già importata come `nx`) per:
1. Identificare il cammino più corto tra Mr. Hi e Officer;
2. Calcolare la lunghezza del cammino più corto.

In [ ]:
# ============== YOUR CODE HERE ==============
raise NotImplementedError
shortest_path = []
lunghezza_shortest_path = None
# ============================================

print("Cammino più corto tra Mr. Hi e Officer:", shortest_path)
print("Lunghezza del cammino più corto tra Mr. Hi e Officer:", lunghezza_shortest_path)

# 6. Utili indici descrittivi <a id=indici> </a>

## 6.1 Indici descrittivi a livello di nodo <a id=indici_nodo> </a>

### Grado di un nodo

* Grado di *i*. Numero di nodi con cui è connesso: $d_i = \sum_{j=1}^{V}Y_{ij}$.

In [ ]:
d_mr_hi = G.degree(mr_hi)

print("Grado del nodo associato a Mr. Hi: {}".format(d_mr_hi))

In [ ]:
plt.hist(dict(G.degree()).values())
plt.title("Istogramma dei gradi")
plt.xlabel('Grado')
plt.ylabel('Frequenza')
plt.show()

### Esercizio

Calcolare il grado del nodo *Mr. Hi* a partire dalla matrice di adiacenza $Y$ (vedi definizione sopra).

In [ ]:
# ============== YOUR CODE HERE ==============
raise NotImplementedError
d_mr_hi = None
# ============================================

print("Grado del nodo associato a Mr. Hi: {}".format(d_mr_hi))

### *Betweenness*

* Livello di *betweenness* di $i$. È la somma (fatta su tutte le coppie di nodi $u$ e $v$ diversi da $i$) del rapporto tra il numero degli *shortest paths* tra $u$ e $v$ che passano per $i$, $n_{uv}(i)$, ed il totale degli *shortest paths* tra $u$ e $v$, $n_{uv}$: $g_i = \sum_{u \neq i \neq u}\frac{n_{uv}(i)}{n_{uv}}$.


[betweenness_centrality](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html): *Compute the shortest-path betweenness centrality for nodes. Betweenness centrality of a node $v$ is the sum of the fraction of all-pairs shortest paths that pass through $v$*.

* **k** (int, optional (default=None)) – If k is not None use k node samples to estimate betweenness. The value of k <= n where n is the number of nodes in the graph. 

In [ ]:
g = nx.betweenness_centrality(G, k=None)
print("Livelli di betweenness per ogni nodo:", g)

### Esercizio

Ricavare a partire dal dizionario `g` il livello di *betweenness* di Mr. Hi.

In [ ]:
# ============== YOUR CODE HERE ==============
raise NotImplementedError
betweenness_mr_hi = None
# ============================================

print("Betweenness del nodo \"Mr. Hi\": {:.2f}".format(betweenness_mr_hi))

In [ ]:
plt.title("Istogramma della betweenness")
plt.hist(g.values())
plt.xlabel('Betweenness')
plt.ylabel('Frequenza')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler((100, 1000))
node_size_betweenness = scaler.fit_transform(np.array([g[nodo] for nodo in N]).reshape(-1, 1))

plt.title("Rappresentazione nella rete dei diversi livelli di betweenness")
nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    node_color=node_color_club,
    node_size=node_size_betweenness
)
plt.axis('off')
plt.tight_layout()
plt.show()

## 6.2 Indici descrittivi a livello di rete <a id=indici_rete> </a>

### Densità del grafo

* Densità di $G$. Frequenza relativa del numero totale di archi osservati, sul totale degli archi possibili: $D = \frac{1}{V(V - 1)}\sum Y_{ij}$;

In [ ]:
D = nx.density(G)

print("Densità del grafo G: {:.2f}".format(D))

## Esercizio
Calcolare la densità del grafo $G$ a partire dalla matrice di adiacenza $Y$ (vedi definizione sopra).

In [ ]:
# ============== YOUR CODE HERE ==============
raise NotImplementedError
D = None
# ============================================

print("Densità del grafo G: {:.2f}".format(D))

### Diametro del grafo

* Diametro di $G$. Lunghezza del più lungo *shortest path*;

In [ ]:
print("Diametro del grafo G: {}".format(nx.diameter(G)))

* Lunghezza media di *shortest path*. Media delle lunghezze minime di *path*. $L = \frac{1}{V(V - 1)}\sum s_{ij}$.

In [ ]:
L = nx.average_shortest_path_length(G)
print("Lunghezza media di shortest path del grafo G: {:.2f}".format(L))

### Misure di omofilia - Modularità

* Modularità. Frazione di archi che connette nodi dello stesso tipo meno il valore atteso della stessa quantità in una rete con connessioni casuali: $Q = \sum_k^K (e_{kk} - a_k^2)$, dove $e_{kk}$ rappresenta la frazione degli archi completamente contenuti nella comunità $k$ e $a_k$ è la frazione delle estremità degli archi contenuti nella comunità $k$.

Nota: Nella rete con connessioni casuali ogni nodo viene vincolato a mantenere il suo grado, in pratica è come se si tagliasse ogni arco in due e ogni mezzo arco, chiamato *stub*, venisse ricablato casualmente con qualsiasi altro *stub* nella rete. Se $a^\star_k$ è il numero di *stub* nella comunità $k$, il numero di possibili archi contenuti nella comunità $k$ (consentendo *self loops*) è ${a^\star_k}^2$, il valore atteso degli archi contenuti nella comunità $k$ è quindi $^{{a^\star}_k^2}/_{l^2}$ dove $l$ è il numero di *stub* nella rete. Visto che $a_k = {^{{a^\star}_k}/_{l}}$, il valore atteso degli archi contenuti nella comunità è anche pari a $a_k^2$.

In [ ]:
G_esem = nx.make_small_graph(["edgelist", "Esempio di rete", 6, [[1, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 6], [2, 5]]])
pos_esem = nx.spring_layout(G_esem, iterations=50, seed=3)
part_esem = {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1}
node_color_esem = ['lightblue' if (part_esem[nodo] == 0) else 'orange' for nodo in G_esem.nodes()]

nx.draw_networkx(
    G_esem, 
    pos=pos_esem,
    node_color=node_color_esem,
    with_labels=True
)
plt.title("Esempio di rete")
plt.axis('off')
plt.tight_layout()

In [ ]:
import community
import pandas as pd

### Calcolo della modularità

In [ ]:
Q = community.modularity(part_esem, G_esem)

print("Modularità del grafo: {:.2f}".format(Q))

### Esempio

Ricavare la modularità di un grafo senza utilizzare la funzione `modularity()`.

In [ ]:
freq_rel = pd.DataFrame(
    [[6 / 14, 1 / 14], [1 / 14, 6 / 14]],
    columns=["Gruppo 1", "Gruppo 2"], 
    index=["Gruppo 1", "Gruppo 2"]
)
freq_rel['Marginale'] = freq_rel.sum(axis=0)
freq_rel = freq_rel.append(pd.Series(freq_rel.sum(axis=0), name='Marginale'))

print("Tabella delle frequenze relative:")
display(freq_rel.round(2))

num_archi = 7
num_estremita = num_archi * 2
num_archi_1 = 3
num_archi_2 = 3
num_estremita_1 = 7
num_estremita_2 = 7

Q_a_mano = (num_archi_1 / num_archi + num_archi_2 / num_archi) - \
    ((num_estremita_1 / num_estremita) ** 2 + (num_estremita_2 / num_estremita) ** 2)

Q_freq_rel = np.diagonal(freq_rel)[:-1].sum() - (freq_rel['Marginale'][:-1] ** 2).sum()

Q = community.modularity(part_esem, G_esem)

print("\nValore di Q calcolato \"a mano\": {:.2f}".format(Q_a_mano))
print("Valore di Q calcolato a partire dalla matrice delle frequenze relative: {:.2f}".format(Q_freq_rel))
print("Valore di Q calcolato tramite la funzione modularity: {:.2f}".format(Q))

### Esercizio

Definire una nuova rete e ripetere l'esercizio.

In [ ]:
# ============== YOUR CODE HERE ==============
raise NotImplementedError
# ============================================

# 7. Community Detection <a id=detection> </a>

**Obiettivo**: Dividere la rete in comunità di nodi, in modo che nodi all'interno di ogni comunità abbiano molte connessioni tra loro (rete densa), mentre nodi in comunità diverse siano poco connessi (rete sparsa). Esistono vari approcci:
* **Algoritmo di Girvan-Newman**: basato sulla *betweenness* di arco;
* **Metodo di Louvain**: ottimizzazione della modularità;
* E altri ...

In [ ]:
communities = next(nx.community.girvan_newman(G))

In [ ]:
part_girvan_newman = {node: int(node in communities[0]) for node in N}

In [ ]:
Q_girvan_newman = community.modularity(part_girvan_newman, G)
print("Modularità, comunità identificate con Girvan-Newman (n = 2): {:.2f}".format(Q_girvan_newman))

## 7.1 Metodo di Louvain <a id=louvain> </a>

**Metodo di Louvain**
1. L'algoritmo è inizializzato mettendo ogni nodo in una comunità diversa;
2. Per ogni nodo $i$ si calcola il guadagno in modularità $\Delta Q_{i:i \rightarrow C_j}$ ottenuto nello spostare $i$ dalla sua comunità a quella di ogni nodo $j$ connesso ad $i$;
3. Il nodo $i$ viene messo nella comunità con maggiore incremento in modularità se l'incremento è positivo. Altrimenti rimane nella sua comunità. Questo processo è applicato in ripetizione e sequenzialmente a tutti i nodi fino a quando la modularità non aumenta più;
4. Le comunità vengono raggruppate a formare una nuova rete (pesata con *self loops*) in cui le comunità sono i nuovi nodi e i nuovi pesi degli archi sono dati dal numero totale di archi che connettono i nodi nelle due comunità;
5. Torna a 2. e riapplica il procedimento alla nuova rete tra comunità.

In [ ]:
part_louvain = community.best_partition(G)

In [ ]:
Q_louvain = community.modularity(part_louvain, G)
print("Modularità, comunità identificate con Louvain: {:.2f}".format(Q_louvain))

In [ ]:
node_color_girvan_newman = ['lightgreen' if (nodo in communities[0]) else 'yellow' for nodo in N]
node_color_louvain = [part_louvain.get(nodo) for nodo in N]

plt.figure(figsize=(12, 8))

plt.subplot(221)

nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    node_color=node_color_club,
    node_size=node_size_betweenness
)
plt.title("I due club")
plt.axis('off')
plt.tight_layout()

plt.subplot(222)

nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    node_color=node_color_girvan_newman,
    node_size=node_size_betweenness
)
plt.title("Comunità trovate usando l'algoritmo di Girvan-Newman (n = 2)")
plt.axis('off')
plt.tight_layout()

plt.subplot(223)
plt.text(0.5, 0.6, "Community Detection", size=30, ha="center", va="center")
plt.axis('off')
plt.tight_layout()

plt.subplot(224)

nx.draw_networkx(
    G, 
    pos=pos,
    with_labels=True,
    cmap=plt.get_cmap("Set2"),
    node_color=node_color_louvain,
    node_size=node_size_betweenness
)
plt.title("Comunità trovate usando il metodo di Louvain")
plt.axis('off')
plt.tight_layout()

plt.show()